# Joining Data in SQL

Now that you've learned the basics of SQL, it's time to supercharge your queries using joins and relational set theory. In this notebook, you'll learn all about the power of joining tables while exploring interesting features of countries and their cities throughout the world. You will master inner and outer joins, as well as self joins, semi joins, anti joins and cross joins—fundamental tools in any PostgreSQL wizard's toolbox. Never fear set theory again after learning all about unions, intersections, and except clauses through easy-to-understand diagrams and examples. Lastly, you'll be introduced to the challenging topic of subqueries. You will be able to visually grasp these ideas by using Venn diagrams and other linking illustrations.

In [1]:
! postgres --version

postgres (PostgreSQL) 11.3


In [2]:
import pandas as pd
import psycopg2 as pg

In [ ]:
# To setup the database:
# In pgAdmin (http://127.0.0.1:50822/browser/): 
# - create a user the same as for macos user
# - create films database with the user as the owner
# In terminal in the countries2 folder:
# > psql -U ksatola countries < 'countries.sql''

In [3]:
conn = pg.connect(database="countries",user="ksatola", password="ksroot")

In [4]:
sql ='''
SELECT * 
FROM countries 
LIMIT 5
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,Afganistan/Afqanestan,Islamic Emirate,Kabul,69.17610,34.5228
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581.0,Nederland,Constitutional Monarchy,Amsterdam,4.89095,52.3738
2,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,Shqiperia,Republic,Tirane,19.81720,41.3317
3,DZA,Algeria,Africa,Northern Africa,2381740.0,1962.0,Al-Jazair/Algerie,Republic,Algiers,3.05097,36.7397
4,ASM,American Samoa,Oceania,Polynesia,199.0,NaN,Amerika Samoa,US Territory,Pago Pago,-170.69100,-14.2846


## Inner join

You'll be working with the `countries` database containing information about the most populous world cities as well as country-level economic data, population data, and geographic data. This countries database also contains information on languages spoken in each country.

Here is the basic syntax for an INNER JOIN, here including all columns in both tables:

In [5]:
sql ='''
SELECT * 
FROM cities
  -- 1. Inner join to countries
  INNER JOIN countries
    -- 2. Match on the country codes
    ON cities.country_code = countries.code;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop,code,name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,Abidjan,CIV,4765000.0,NaN,4765000.0,CIV,Cote d'Ivoire,Africa,Western Africa,322463.0,1960,Cote dIvoire,Republic,Yamoussoukro,-4.03050,5.33200
1,Abu Dhabi,ARE,1145000.0,NaN,1145000.0,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971,Al-Imarat al-´Arabiya al-Muttahida,Emirate Federation,Abu Dhabi,54.37050,24.47640
2,Abuja,NGA,1235880.0,6000000.0,1235880.0,NGA,Nigeria,Africa,Western Africa,923768.0,1960,Nigeria,Federal Republic,Abuja,7.48906,9.05804
3,Accra,GHA,2070460.0,4010050.0,2070460.0,GHA,Ghana,Africa,Western Africa,238533.0,1957,Ghana,Republic,Accra,-0.20795,5.57045
4,Addis Ababa,ETH,3103670.0,4567860.0,3103670.0,ETH,Ethiopia,Africa,Eastern Africa,1104300.0,-1000,YeItyop´iya,Republic,Addis Ababa,38.74680,9.02274


In [8]:
sql ='''
-- 1. Select name fields (with alias) and region 
SELECT cities.name AS city,
countries.name AS country,
countries.region
FROM cities
INNER JOIN countries
ON cities.country_code = countries.code
'''
df = pd.read_sql(sql, conn)
df.head()

,city,country,region
0,Abidjan,Cote d'Ivoire,Western Africa
1,Abu Dhabi,United Arab Emirates,Middle East
2,Abuja,Nigeria,Western Africa
3,Accra,Ghana,Western Africa
4,Addis Ababa,Ethiopia,Eastern Africa


## Inner join (2)

Instead of writing the full table name, you can use table aliasing as a shortcut. For tables you also use `AS` to add the alias immediately after the table name with a space. Check out the aliasing of cities and countries below.

Notice that to select a field in your query that appears in multiple tables, you'll need to identify which table/table alias you're referring to by using a `.` in your `SELECT` statement.

You'll now explore a way to get data from both the countries and economies tables to examine the inflation rate for both 2010 and 2015.

Sometimes it's easier to write SQL code out of order: you write the `SELECT` statement after you've done the `JOIN`.

- Join the tables `countries` (left) and `economies` (right) aliasing countries AS `c` and economies AS `e`.
- Specify the field to match the tables ON.
- From this join, SELECT:
    - c.code, aliased as country_code.
    - name, year, and inflation_rate, not aliased.

In [9]:
sql ='''
-- 3. Select fields with aliases
SELECT c.code AS country_code, c.name, e.year, inflation_rate
FROM countries AS c
  -- 1. Join to economies (alias e)
  INNER JOIN economies AS e
    -- 2. Match on code
    ON c.code = e.code;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,name,year,inflation_rate
0,AFG,Afghanistan,2010,2.179
1,AFG,Afghanistan,2015,-1.549
2,AGO,Angola,2010,14.480
3,AGO,Angola,2015,10.287
4,ALB,Albania,2010,3.605


## Inner join (3)

The ability to combine multiple joins in a single query is a powerful feature of SQL, e.g:

As you can see here it becomes tedious to continually write long table names in joins. This is when it becomes useful to alias each table using the first letter of its name (e.g. countries AS c)! It is standard practice to alias in this way and, if you choose to alias tables or are asked to specifically for an exercise in this course, you should follow this protocol.

Now, for each country, you want to get the country name, its region, and the fertility rate and unemployment rate for both 2010 and 2015.

Note that results should work throughout this course with or without table aliasing unless specified differently.

- Inner join countries (left) and populations (right) on the code and country_code fields respectively.
- Alias countries AS c and populations AS p.
- Select code, name, and region from countries and also select year and fertility_rate from populations (5 fields in total).

In [10]:
sql ='''
-- 4. Select fields
SELECT c.code, c.name, c.region, p.year, p.fertility_rate
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join with populations (as p)
  INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,region,year,fertility_rate
0,ABW,Aruba,Caribbean,2010,1.704
1,ABW,Aruba,Caribbean,2015,1.647
2,AFG,Afghanistan,Southern and Central Asia,2010,5.746
3,AFG,Afghanistan,Southern and Central Asia,2015,4.653
4,AGO,Angola,Central Africa,2010,6.416


- Add an additional inner join with economies to your previous query by joining on code.
- Include the unemployment_rate column that became available through joining with economies.
- Note that year appears in both populations and economies, so you have to explicitly use e.year instead of year as you did before.

In [11]:
sql ='''
-- 6. Select fields
SELECT c.code, name, region, e.year, fertility_rate, e.unemployment_rate
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join to populations (as p)
  INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
  -- 4. Join to economies (as e)
  INNER JOIN economies AS e
    -- 5. Match on country code
    ON c.code = e.code;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,region,year,fertility_rate,unemployment_rate
0,AFG,Afghanistan,Southern and Central Asia,2015,5.746,NaN
1,AFG,Afghanistan,Southern and Central Asia,2010,5.746,NaN
2,AFG,Afghanistan,Southern and Central Asia,2015,4.653,NaN
3,AFG,Afghanistan,Southern and Central Asia,2010,4.653,NaN
4,AGO,Angola,Central Africa,2015,6.416,NaN


- The trouble with doing your last join on c.code = e.code and not also including year is that e.g. the 2010 value for fertility_rate is also paired with the 2015 value for unemployment_rate.
- Fix your previous query: in your last ON clause, use AND to add an additional joining condition. In addition to joining on code in c and e, also join on year in e and p.

In [12]:
sql ='''
-- 6. Select fields
SELECT c.code, name, region, e.year, fertility_rate, unemployment_rate
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join to populations (as p)
  INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
  -- 4. Join to economies (as e)
  INNER JOIN economies AS e
    -- 5. Match on country code and year
    ON c.code = e.code AND p.year = e.year;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,region,year,fertility_rate,unemployment_rate
0,AFG,Afghanistan,Southern and Central Asia,2010,5.746,NaN
1,AFG,Afghanistan,Southern and Central Asia,2015,4.653,NaN
2,AGO,Angola,Central Africa,2010,6.416,NaN
3,AGO,Angola,Central Africa,2015,5.996,NaN
4,ALB,Albania,Southern Europe,2010,1.663,14.0


## Inner join with using

When joining tables with a common field name, e.g.

You can use USING as a shortcut:

You'll now explore how this can be done with the countries and languages tables.

- Inner join countries on the left and languages on the right with USING(code).
- Select the fields corresponding to:
    - country name AS country,
    - continent name,
    - language name AS language, and
    - whether or not the language is official.
    
Remember to alias your tables using the first letter of their names.

In [16]:
sql ='''
-- 4. Select fields
SELECT c.name AS country, c.continent, l.name AS language, l.official
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join to languages (as l)
  INNER JOIN languages AS l
    -- 3. Match using code
    USING(code);
'''
df = pd.read_sql(sql, conn)
df.head()

,country,continent,language,official
0,Afghanistan,Asia,Dari,True
1,Afghanistan,Asia,Pashto,True
2,Afghanistan,Asia,Turkic,False
3,Afghanistan,Asia,Other,False
4,Albania,Europe,Albanian,True


## Self-join

In this exercise, you'll use the populations table to perform a self-join to calculate the percentage increase in population from 2010 to 2015 for each country code!

Since you'll be joining the populations table to itself, you can alias populations as p1 and also populations as p2. This is good practice whenever you are aliasing and your tables have the same first letter. Note that you are required to alias the tables with self-joins.

- Join populations with itself ON country_code.
- Select the country_code from p1 and the size field from both p1 and p2. SQL won't allow same-named fields, so alias p1.size as size2010 and p2.size as size2015.

In [17]:
sql ='''
-- 4. Select fields with aliases
SELECT p1.country_code, 
       p1.size AS size2010,
       p2.size AS size2015
-- 1. From populations (alias as p1)
FROM populations AS p1
  -- 2. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    -- 3. Match on country code
    ON  p1.country_code = p2.country_code;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size2010,size2015
0,ABW,101597.0,103889.0
1,ABW,101597.0,101597.0
2,ABW,103889.0,103889.0
3,ABW,103889.0,101597.0
4,AFG,27962200.0,32526600.0


Notice from the result that for each country_code you have four entries laying out all combinations of 2010 and 2015.

- Extend the ON in your query to include only those records where the p1.year (2010) matches with p2.year - 5 (2015 - 5 = 2010). This will omit the three entries per country_code that you aren't interested in.

In [18]:
sql ='''
-- 5. Select fields with aliases
SELECT p1.country_code,
       p1.size AS size2010,
       p2.size AS size2015
-- 1. From populations (alias as p1)
FROM populations as p1
  -- 2. Join to itself (alias as p2)
  INNER JOIN populations as p2
    -- 3. Match on country code
    ON p1.country_code = p2.country_code
        -- 4. and year (with calculation)
        AND p1.year = p2.year - 5;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size2010,size2015
0,ABW,101597.0,103889.0
1,AFG,27962200.0,32526600.0
2,AGO,21220000.0,25022000.0
3,ALB,2913020.0,2889170.0
4,AND,84419.0,70473.0


As you just saw, you can also use SQL to calculate values like p2.year - 5 for you. With two fields like size2010 and size2015, you may want to determine the percentage increase from one field to the next:

With two numeric fields A and B, the percentage growth from A to B can be calculated as (B−A)/A∗100.0.

Add a new field to SELECT, aliased as growth_perc, that calculates the percentage population growth from 2010 to 2015 for each country, using p2.size and p1.size.

In [19]:
sql ='''
SELECT p1.country_code,
       p1.size AS size2010, 
       p2.size AS size2015,
       -- 1. calculate growth_perc
       ((p2.size - p1.size)/p1.size * 100.0) AS growth_perc
-- 2. From populations (alias as p1)
FROM populations AS p1
  -- 3. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    -- 4. Match on country code
    ON p1.country_code = p2.country_code
        -- 5. and year (with calculation)
        AND p1.year = p2.year - 5;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size2010,size2015,growth_perc
0,ABW,101597.0,103889.0,2.255972
1,AFG,27962200.0,32526600.0,16.323297
2,AGO,21220000.0,25022000.0,17.917192
3,ALB,2913020.0,2889170.0,-0.818875
4,AND,84419.0,70473.0,-16.519977


## Case when and then

Often it's useful to look at a numerical field not as raw data, but instead as being in different categories or groups.

You can use `CASE` with `WHEN`, `THEN`, `ELSE`, and `END` to define a new grouping field.

Using the countries table, create a new field AS geosize_group that groups the countries into three groups:

- If surface_area is greater than 2 million, geosize_group is 'large'.
- If surface_area is greater than 350 thousand but not larger than 2 million, geosize_group is 'medium'.
- Otherwise, geosize_group is 'small'.

In [20]:
sql ='''
SELECT name, continent, code, surface_area,
    -- 1. First case
    CASE WHEN surface_area > 2000000 THEN 'large'
        -- 2. Second case
        WHEN surface_area > 350000 THEN 'medium'
        -- 3. Else clause + end
        ELSE 'small' END
        -- 4. Alias name
        AS geosize_group
-- 5. From table
FROM countries;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,continent,code,surface_area,geosize_group
0,Afghanistan,Asia,AFG,652090.0,medium
1,Netherlands,Europe,NLD,41526.0,small
2,Albania,Europe,ALB,28748.0,small
3,Algeria,Africa,DZA,2381740.0,large
4,American Samoa,Oceania,ASM,199.0,small


## Inner challenge

The table you created with the added geosize_group field has been loaded for you here with the name countries_plus. Observe the use of (and the placement of) the INTO command to create this countries_plus table:

You will now explore the relationship between the size of a country in terms of surface area and in terms of population using grouping fields created with CASE.

By the end of this exercise, you'll be writing two queries back-to-back in a single script. You got this!

Using the populations table focused only for the year 2015, create a new field AS popsize_group to organize population size into

- 'large' (> 50 million),
- 'medium' (> 1 million), and
- 'small' groups.

Select only the country code, population size, and this new popsize_group as fields.

In [21]:
sql ='''
SELECT country_code, size,
    -- 1. First case
    CASE WHEN size > 50000000 THEN 'large'
        -- 2. Second case
        WHEN size > 1000000 THEN 'medium'
        -- 3. Else clause + end
        ELSE 'small' END
        -- 4. Alias name
        AS popsize_group
-- 5. From table
FROM populations
-- 6. Focus on 2015
WHERE year = 2015;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size,popsize_group
0,ABW,103889.0,small
1,AFG,32526600.0,medium
2,AGO,25022000.0,medium
3,ALB,2889170.0,medium
4,AND,70473.0,small


- Use INTO to save the result of the previous query as pop_plus. You can see an example of this in the countries_plus code in the assignment text. Make sure to include a ; at the end of your WHERE clause!

- Then, include another query below your first query to display all the records in pop_plus using SELECT * FROM pop_plus; so that you generate results and this will display pop_plus in query result.

In [22]:
sql ='''
SELECT country_code, size,
    CASE WHEN size > 50000000 THEN 'large'
        WHEN size > 1000000 THEN 'medium'
        ELSE 'small' END
        AS popsize_group
-- 1. Into table
INTO pop_plus
FROM populations
WHERE year = 2015;

-- 2. Select all columns of pop_plus
SELECT * FROM pop_plus;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size,popsize_group
0,ABW,103889.0,small
1,AFG,32526600.0,medium
2,AGO,25022000.0,medium
3,ALB,2889170.0,medium
4,AND,70473.0,small


- Keep the first query intact that creates pop_plus using INTO.
- Write a query to join countries_plus AS c on the left with pop_plus AS p on the right matching on the country code fields.
- Sort the data based on geosize_group, in ascending order so that large appears on top.
- Select the name, continent, geosize_group, and popsize_group fields.

In [24]:
sql ='''
SELECT country_code, size,
  CASE WHEN size > 50000000
            THEN 'large'
       WHEN size > 1000000
            THEN 'medium'
       ELSE 'small' END
       AS popsize_group
INTO pop_plus2       
FROM populations
WHERE year = 2015;

-- 5. Select fields
SELECT c.name, c.continent, c.geosize_group, p.popsize_group
-- 1. From countries_plus (alias as c)
FROM countries_plus as c
  -- 2. Join to pop_plus (alias as p)
  INNER JOIN pop_plus2 as p
    -- 3. Match on country code
    ON c.code = p.country_code
-- 4. Order the table    
ORDER BY c.geosize_group;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,continent,geosize_group,popsize_group
0,India,Asia,large,large
1,United States,North America,large,large
2,Saudi Arabia,Asia,large,medium
3,China,Asia,large,large
4,Kazakhstan,Asia,large,medium


## Left Join

Now you'll explore the differences between performing an `inner join` and a `left join` using the cities and countries tables.

You'll begin by performing an inner join with the cities table on the left and the countries table on the right. Remember to alias the name of the city field as city and the name of the country field as country.

You will then change the query to a left join. Take note of how many records are in each query here!

In [25]:
sql ='''
-- Select the city name (with alias), the country code,
-- the country name (with alias), the region,
-- and the city proper population
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
-- From left table (with alias)
FROM cities AS c1
  -- Join to right table (with alias)
  INNER JOIN countries AS c2
    -- Match on country code
    ON c1.country_code = c2.code
-- Order by descending country code
ORDER BY code DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,city,code,country,region,city_proper_pop
0,Harare,ZWE,Zimbabwe,Eastern Africa,1606000.0
1,Lusaka,ZMB,Zambia,Eastern Africa,1742980.0
2,Cape Town,ZAF,South Africa,Southern Africa,3740030.0
3,Johannesburg,ZAF,South Africa,Southern Africa,4434830.0
4,Durban,ZAF,South Africa,Southern Africa,3442360.0


Change the code to perform a LEFT JOIN instead of an INNER JOIN. After executing this query, note how many records the query result contains.

In [26]:
sql ='''
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
FROM cities AS c1
  -- 1. Join right table (with alias)
  LEFT JOIN countries AS c2
    -- 2. Match on country code
    ON c1.country_code = c2.code
-- 3. Order by descending country code
ORDER BY code DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,city,code,country,region,city_proper_pop
0,Taichung,None,None,None,2752410.0
1,Tainan,None,None,None,1885250.0
2,Kaohsiung,None,None,None,2778920.0
3,Bucharest,None,None,None,1883420.0
4,Taipei,None,None,None,2704970.0


Notice that the INNER JOIN version resulted in 230 records. The LEFT JOIN version returned 236 rows.

## Left join (2)

Next, you'll try out another example comparing an inner join to its corresponding left join. Before you begin though, take note of how many records are in both the countries and languages tables below.

You will begin with an inner join on the countries table on the left with the languages table on the right. Then you'll change the code to a left join in the next bullet.

Note the use of multi-line comments here using /* and */.

- Perform an inner join. Alias the name of the country field as country and the name of the language field as language.
- Sort based on descending country name.

In [27]:
sql ='''
/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  INNER JOIN languages AS l
    -- 3. Match on fields
    ON c.code = l.code
-- 4. Order by descending country
ORDER BY country DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,local_name,language,percent
0,Zimbabwe,Zimbabwe,Shona,NaN
1,Zimbabwe,Zimbabwe,Tonga,NaN
2,Zimbabwe,Zimbabwe,Tswana,NaN
3,Zimbabwe,Zimbabwe,Venda,NaN
4,Zimbabwe,Zimbabwe,Xhosa,NaN


- Perform a left join instead of an inner join. Observe the result, and also note the change in the number of records in the result.
- Carefully review which records appear in the left join result, but not in the inner join result.

In [28]:
sql ='''
/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  LEFT JOIN languages AS l
    -- 3. Match on fields
    ON c.code = l.code
-- 4. Order by descending country
ORDER BY country DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,local_name,language,percent
0,Zimbabwe,Zimbabwe,Chibarwe,NaN
1,Zimbabwe,Zimbabwe,Shona,NaN
2,Zimbabwe,Zimbabwe,Ndebele,NaN
3,Zimbabwe,Zimbabwe,English,NaN
4,Zimbabwe,Zimbabwe,Chewa,NaN


Notice that the INNER JOIN version resulted in 914 records. The LEFT JOIN version returned 921 rows.

## Left join (3)

You'll now revisit the use of the `AVG()` function. You will use it in combination with left join to determine the average gross domestic product (GDP) per capita by region in 2010.

- Begin with a left join with the countries table on the left and the economies table on the right.
- Focus only on records with 2010 as the year.

In [29]:
sql ='''
-- 5. Select name, region, and gdp_percapita
SELECT name, region, gdp_percapita
-- 1. From countries (alias as c)
FROM countries AS c
  -- 2. Left join with economies (alias as e)
  LEFT JOIN economies AS e
    -- 3. Match on code fields
    ON c.code = e.code
-- 4. Focus on 2010
WHERE e.year = 2010;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,region,gdp_percapita
0,Afghanistan,Southern and Central Asia,539.667
1,Angola,Central Africa,3599.270
2,Albania,Southern Europe,4098.130
3,United Arab Emirates,Middle East,34628.600
4,Argentina,South America,10413.000


- Modify your code to calculate the average GDP per capita AS avg_gdp for each region in 2010.
- Select the region and avg_gdp fields.

In [30]:
sql ='''
-- Select fields
SELECT region, AVG(gdp_percapita) as avg_gdp
-- From countries (alias as c)
FROM countries AS c
  -- Left join with economies (alias as e)
  LEFT JOIN economies AS e
    -- Match on code fields
    ON c.code = e.code
-- Focus on 2010
WHERE year = 2010
-- Group by region
GROUP BY region;
'''
df = pd.read_sql(sql, conn)
df.head()

,region,avg_gdp
0,Southern Africa,5051.597974
1,Caribbean,11413.339454
2,Eastern Africa,1757.348162
3,Southern Europe,22926.410911
4,Eastern Asia,26205.851400


Arrange this data on average GDP per capita for each region in 2010 from highest to lowest average GDP per capita.

In [31]:
sql ='''
-- Select fields
SELECT region, AVG(gdp_percapita) AS avg_gdp
-- From countries (alias as c)
FROM countries AS c
  -- Left join with economies (alias as e)
  LEFT JOIN economies AS e
    -- Match on code fields
    ON c.code = e.code
-- Focus on 2010
WHERE year = 2010
-- Group by region
GROUP BY region
-- Order by descending avg_gdp
ORDER BY avg_gdp DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,region,avg_gdp
0,Western Europe,58130.961496
1,Nordic Countries,57073.997656
2,North America,47911.509766
3,Australia and New Zealand,44792.384766
4,British Islands,43588.330078


## Right join

Right joins aren't as common as left joins. One reason why is that you can always write a right join as a left join.

The left join code is commented out here. Your task is to write a new query using rights joins that produces the same result as what the query using left joins produces. Keep this left joins code commented as you write your own query just below it using right joins to solve the problem.

Note the order of the joins matters in your conversion to using right joins!

In [32]:
sql ='''
-- convert this code to use RIGHT JOINs instead of LEFT JOINs
/*
SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM cities
  LEFT JOIN countries
    ON cities.country_code = countries.code
  LEFT JOIN languages
    ON countries.code = languages.code
ORDER BY city, language;
*/

SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM languages
  RIGHT JOIN countries
    ON countries.code = languages.code
  RIGHT JOIN cities
    ON cities.country_code = countries.code
ORDER BY city, language;
'''
df = pd.read_sql(sql, conn)
df.head()

,city,urbanarea_pop,country,indep_year,language,percent
0,Abidjan,4765000.0,Cote d'Ivoire,1960.0,French,NaN
1,Abidjan,4765000.0,Cote d'Ivoire,1960.0,Other,NaN
2,Abu Dhabi,1145000.0,United Arab Emirates,1971.0,Arabic,NaN
3,Abu Dhabi,1145000.0,United Arab Emirates,1971.0,English,NaN
4,Abu Dhabi,1145000.0,United Arab Emirates,1971.0,Hindi,NaN


## Full join

In this exercise, you'll examine how your results differ when using a full join versus using a left join versus using an inner join with the countries and currencies tables.

You will focus on the North American region and also where the name of the country is missing. Dig in to see what we mean!

Begin with a full join with countries on the left and currencies on the right. The fields of interest have been SELECTed for you throughout this exercise.

Then complete a similar left join and conclude with an inner join.

Choose records in which region corresponds to North America or is NULL.

In [33]:
sql ='''
SELECT name AS country, code, region, basic_unit
-- 3. From countries
FROM countries
  -- 4. Join to currencies
  FULL JOIN currencies
    -- 5. Match on code
    USING (code)
-- 1. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
-- 2. Order by region
ORDER BY region;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,code,region,basic_unit
0,Canada,CAN,North America,Canadian dollar
1,United States,USA,North America,United States dollar
2,Bermuda,BMU,North America,Bermudian dollar
3,Greenland,GRL,North America,None
4,None,TMP,None,United States dollar


Repeat the same query as above but use a LEFT JOIN instead of a FULL JOIN. Note what has changed compared to the FULL JOIN result!

In [34]:
sql ='''
SELECT name AS country, code, region, basic_unit
-- 1. From countries
FROM countries
  -- 2. Join to currencies
  LEFT JOIN currencies
    -- 3. Match on code
    USING (code)
-- 4. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
-- 5. Order by region
ORDER BY region;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,code,region,basic_unit
0,Bermuda,BMU,North America,Bermudian dollar
1,Canada,CAN,North America,Canadian dollar
2,United States,USA,North America,United States dollar
3,Greenland,GRL,North America,None


Repeat the same query as above but use an INNER JOIN instead of a FULL JOIN. Note what has changed compared to the FULL JOIN and LEFT JOIN results!

In [35]:
sql ='''
SELECT name AS country, code, region, basic_unit
FROM countries
  -- 1. Join to currencies
  INNER JOIN currencies
    USING (code)
-- 2. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
-- 3. Order by region
ORDER BY region;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,code,region,basic_unit
0,Bermuda,BMU,North America,Bermudian dollar
1,Canada,CAN,North America,Canadian dollar
2,United States,USA,North America,United States dollar


Have you kept an eye out on the different numbers of records these queries returned? The FULL JOIN query returned 17 rows, the OUTER JOIN returned 4 rows, and the INNER JOIN only returned 3 rows. Do these results make sense to you?

## Full join (2)

You'll now investigate a similar exercise to the last one, but this time focused on using a table with more records on the left than the right. You'll work with the languages and countries tables.

Begin with a full join with languages on the left and countries on the right. Appropriate fields have been selected for you again here.

- Choose records in which countries.name starts with the capital letter 'V' or is NULL.
- Arrange by countries.name in ascending order to more clearly see the results.

In [36]:
sql ='''
SELECT countries.name, code, languages.name AS language
-- 3. From languages
FROM languages
  -- 4. Join to countries
  FULL JOIN countries
    -- 5. Match on code
    USING (code)
-- 1. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
-- 2. Order by ascending countries.name
ORDER BY countries.name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,code,language
0,Vanuatu,VUT,Tribal Languages
1,Vanuatu,VUT,English
2,Vanuatu,VUT,French
3,Vanuatu,VUT,Other
4,Vanuatu,VUT,Bislama


Repeat the same query as above but use a left join instead of a full join. Note what has changed compared to the full join result!

In [37]:
sql ='''
SELECT countries.name, code, languages.name AS language
FROM languages
  -- 1. Join to countries
  LEFT JOIN countries
    -- 2. Match using code
    USING (code)
-- 3. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
ORDER BY countries.name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,code,language
0,Vanuatu,VUT,English
1,Vanuatu,VUT,Other
2,Vanuatu,VUT,French
3,Vanuatu,VUT,Tribal Languages
4,Vanuatu,VUT,Bislama


Repeat once more, but use an inner join instead of a left join. Note what has changed compared to the full join and left join results.

In [38]:
sql ='''
SELECT countries.name, code, languages.name AS language
FROM languages
  -- 1. Join to countries
  INNER JOIN countries
    USING (code)
-- 2. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
ORDER BY countries.name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,code,language
0,Vanuatu,VUT,Tribal Languages
1,Vanuatu,VUT,Bislama
2,Vanuatu,VUT,English
3,Vanuatu,VUT,French
4,Vanuatu,VUT,Other


## Full join (3)

You'll now explore using two consecutive full joins on the three tables you worked with in the previous two exercises.

- Complete a full join with countries on the left and languages on the right.
- Next, full join this result with currencies on the right.
- Use LIKE to choose the Melanesia and Micronesia regions (Hint: 'M%esia').
- Select the fields corresponding to the country name AS country, region, language name AS language, and basic and fractional units of currency.

In [39]:
sql ='''
-- 7. Select fields (with aliases)
SELECT c1.name AS country, region, l.name AS language,
       basic_unit, frac_unit
-- 1. From countries (alias as c1)
FROM countries AS c1
  -- 2. Join with languages (alias as l)
  FULL JOIN languages AS l
    -- 3. Match on code
    USING (code)
  -- 4. Join with currencies (alias as c2)
  FULL JOIN currencies AS c2
    -- 5. Match on code
    USING (code)
-- 6. Where region like Melanesia and Micronesia
WHERE region LIKE 'M%esia';
'''
df = pd.read_sql(sql, conn)
df.head()

,country,region,language,basic_unit,frac_unit
0,Kiribati,Micronesia,English,Australian dollar,Cent
1,Kiribati,Micronesia,Kiribati,Australian dollar,Cent
2,Marshall Islands,Micronesia,Other,United States dollar,Cent
3,Marshall Islands,Micronesia,Marshallese,United States dollar,Cent
4,Nauru,Micronesia,Other,Australian dollar,Cent


## A table of two cities

This exercise looks to explore languages potentially and most frequently spoken in the cities of Hyderabad, India and Hyderabad, Pakistan.

You will begin with a cross join with cities AS c on the left and languages AS l on the right. Then you will modify the query using an inner join in the next tab.

- Create the cross join as described above. (Recall that cross joins do not use ON or USING.)
- Make use of LIKE and Hyder% to choose Hyderabad in both countries.
- Select only the city name AS city and language name AS language.

In [40]:
sql ='''
-- 4. Select fields
SELECT c.name AS city, l.name AS language
-- 1. From cities (alias as c)
FROM cities AS c        
  -- 2. Join to languages (alias as l)
  CROSS JOIN languages AS l
-- 3. Where c.name like Hyderabad
WHERE c.name LIKE 'Hyder%';
'''
df = pd.read_sql(sql, conn)
df.head()

,city,language
0,Hyderabad (India),Dari
1,Hyderabad,Dari
2,Hyderabad (India),Pashto
3,Hyderabad,Pashto
4,Hyderabad (India),Turkic


- Use an inner join instead of a cross join. Think about what the difference will be in the results for this inner join result and the one for the cross join.

In [41]:
sql ='''
-- 5. Select fields
SELECT c.name AS city, l.name AS language
-- 1. From cities (alias as c)
FROM cities AS c      
  -- 2. Join to languages (alias as l)
  INNER JOIN languages AS l
    -- 3. Match on country code
    ON c.country_code = l.code
-- 4. Where c.name like Hyderabad
WHERE c.name LIKE 'Hyder%';
'''
df = pd.read_sql(sql, conn)
df.head()

,city,language
0,Hyderabad (India),Hindi
1,Hyderabad (India),Bengali
2,Hyderabad (India),Telugu
3,Hyderabad (India),Marathi
4,Hyderabad (India),Tamil


## Outer challenge

Now that you're fully equipped to use outer joins, try a challenge problem to test your knowledge!

In terms of life expectancy for 2010, determine the names of the lowest five countries and their regions.

- Select country name AS country, region, and life expectancy AS life_exp.
- Make sure to use LEFT JOIN, WHERE, ORDER BY, and LIMIT.

In [42]:
sql ='''
-- Select fields
SELECT c.name as country, region, p.life_expectancy as life_exp
-- From countries (alias as c)
FROM countries AS c
  -- Join to populations (alias as p)
  LEFT JOIN populations AS p
    -- Match on country code
    ON c.code = p.country_code
-- Focus on 2010
WHERE p.year = 2010
-- Order by life_exp
ORDER BY life_exp
-- Limit to 5 records
LIMIT 5;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,region,life_exp
0,Lesotho,Southern Africa,47.4834
1,Central African Republic,Central Africa,47.6253
2,Sierra Leone,Western Africa,48.2290
3,Swaziland,Southern Africa,48.3458
4,Zimbabwe,Eastern Africa,49.5747
